# Exercise 4a
#### Lars Kühmichel, Nicolas Wolf

## 2 Red Cards Study

### 2.1 Loading and Cleaning the Data

#### What do the feature names stand for?

What the feature names stand for can be found in the `README.txt`.
E.g. `games` stands for "number of games in the player-referee",
i.e. how many games that player played under that referee.

#### Which irrelevant features might be dropped?
Features that are irrelevant are for example `birthday`, `height` or `weight`.
Features like `games` and `redCards` are essential to the study,
but even features like `refNum` are important to adjust
for individual referee bias.

#### What relevant features might be missing, but can be computed?

One example is the total number of red cards, given by `yellowReds + redCards`.


#### Are there missing data values, and how should they be dealt with?

Some features, especially skin color ratings, are occasionally missing. We will disaggregate the data to
deal with these in a simpler way. We drop dyads that have missing color ratings.



In [24]:
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from uncertainties import ufloat
import seaborn as sns


#### Disaggregating the data

The data is more usable in a format that has each instance (line) as a single player-ref interaction.
See [this notebook](https://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb).

In [26]:
def disaggregate(filepath, out_path):
    """
    This code was copied and converted to Python 3 from disaggregate_v3.py
    Some of it is of questionable quality. Blame the original devs.
    see https://osf.io/w7tds/
    """
    dataframe = pd.read_csv(filepath)

    # add new vars
    dataframe["skintone"] = (dataframe["rater1"] + dataframe["rater2"]) / 2
    dataframe["all_reds"] = dataframe["yellowReds"] + dataframe["redCards"]
    dataframe["all_reds_strict"] = dataframe["redCards"]
    dataframe["ref_count"] = 0

    # add a column which tracks how many games each ref is involved in
    refs = pd.unique(dataframe["refNum"].values.ravel())  # list all unique ref IDs

    # for each ref, count their dyads
    print("Counting dyads")
    for r in refs:
        m = dataframe["refNum"] == r
        dataframe["ref_count"][m] = len(dataframe[m])

    colnames = list(dataframe.columns)

    print("creating out")
    j = 0
    out = [0 for _ in range(sum(dataframe["games"]))]
    for _, row in dataframe.iterrows():
        n = row["games"]
        c = row["all_reds"]
        d = row["all_reds_strict"]

        for _ in range(n):
            row["all_reds"] = 1 if (c-_) > 0 else 0
            row["all_reds_strict"] = 1 if (d-_) > 0 else 0
            rowlist = list(row)
            out[j] = rowlist
            j += 1
            if j % 10000 == 0:
                print(f"Number {j} of {dataframe.shape[0]}")

    pd.DataFrame(out, columns=colnames).to_csv(out_path, index=False)


filepath = Path("data", "dataset", "crowdstorm_disaggregated.csv")

if not filepath.is_file():
    print("Creating disaggregated dataset...")
    p = Path("data", "dataset", "CrowdstormingDataJuly1st.csv")
    disaggregate(p, filepath)

if not filepath.is_file():
    raise RuntimeError("Could not create disaggregated dataset.")

dataframe = pd.read_csv(filepath)

# show some of the dataframe
dataframe.iloc[0:6,0:13]


,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,1,0
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,1,0
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,0,0
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,0,0
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,0,0
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,1,0


#### Transforming categorical features to a one-hot encoding

When using the code we decided this is not useful and went with UIDs instead.

```

def one_hot(a):
    """
    See https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array
    :param a: Array of categorical features (e.g. league strings)
    """
    _, i = np.unique(a, return_inverse=True)
    result = np.full((i.size, i.max() + 1), False)
    result[np.arange(i.size),i] = True

    return result

# for some reason pandas only accepts these as 2d if they are lists, not numpy arrays
dataframe.club = list(one_hot(dataframe.club.fillna("")))
dataframe.leagueCountry = list(one_hot(dataframe.leagueCountry.fillna("")))
dataframe.position = list(one_hot(dataframe.position.fillna("")))

# print some samples to show this worked
print("club")
print(dataframe.club[0])
print("leagueCountry")
print(dataframe.leagueCountry[0])
print("position")
print(dataframe.position[0])

```

#### Transforming categorical features to unique IDs

In [ ]:
def uid(a):
    _, i = np.unique(a, return_inverse=True)
    return i

dataframe.club = list(uid(dataframe.club.fillna("")))
dataframe.leagueCountry = list(uid(dataframe.leagueCountry.fillna("")))
dataframe.position = list(uid(dataframe.position.fillna("")))

# print some samples to show this worked
print("club")
print(dataframe.club[0])
print("leagueCountry")
print(dataframe.leagueCountry[0])
print("position")
print(dataframe.position[0])

#### Distribution of individual ref-player dyad game numbers

See again [this notebook](https://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb#What-is-the-distribution-of-individual-ref-player-dyad-game-numbers?).

In [ ]:
# distribution of individual ref-player dyad game numbers
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(dataframe.games, bins=max(dataframe.games), histtype="stepfilled")
axes[0].set_xlabel("Number of Interactions")
axes[0].set_ylabel("Frequency")

axes[1].hist(dataframe.games, bins=max(dataframe.games), histtype="stepfilled")
axes[1].set_yscale("symlog")
axes[1].set_title("Log Scaled")
axes[1].set_xlabel("Number of Interactions")
axes[1].set_ylabel("Log Frequency")

fig.suptitle("Distribution of Individual Ref-Player Dyad Game Numbers")
fig.tight_layout()
plt.show()

#### Do the Raters agree on skin colour ratings?

The raters agree with one another in $76.8\%$ of cases.
When they disagree, they still typically pick neighbouring categories.
It is therefore useful to take the mean of the two raters since they disagree relatively often.

In [ ]:
rater1 = np.array(dataframe.rater1)
rater2 = np.array(dataframe.rater2)

# remove nan values. Here, it is important to remove associated dyads from both raters
mask1 = ~np.isnan(rater1)
mask2 = ~np.isnan(rater2)
mask = mask1 & mask2
rater1 = rater1[mask]
rater2 = rater2[mask]

nagree = np.sum(rater1 == rater2)
diff = rater1 - rater2

print(f"Number of times raters agree: {nagree}")
print(f"Number of times raters disagree: {len(diff) - nagree}")
print(f"Raters agree in {100 * nagree / len(diff):.2f}% of cases.")

_df = pd.DataFrame(pd.Series(diff).value_counts(), columns=["Counts"])
_df.index.name = "Difference"

_df

#### Comparison of Raters

In [ ]:
# bar charts are slow (because they need np.unique counts),
# so we will use a hack to generate something equivalent
values = np.array([0.0, 0.25, 0.50, 0.75, 1.0])
width = 0.05
epsilon = 0.000001

bins1 = np.array(list(zip(values - width, values + epsilon))).flatten()
bins2 = np.array(list(zip(values - epsilon, values + width))).flatten()

plt.figure(figsize=(10, 6))
plt.hist(dataframe.rater1, bins=bins1, label="Rater 1")
plt.hist(dataframe.rater2, bins=bins2, label="Rater 2")
plt.xticks(values)
plt.xlabel("Rating")
plt.ylabel("Frequency")
plt.legend()
plt.title("Comparison of Raters")
plt.show()

#### Where do raters disagree most?

From the plots it is evident that the two raters disagree most on lighter skin colors. The most common case
where they disagree has Rater 1 select the skintone as "Very Light" $(0.0)$ and Rater 2 as "Light" $(0.25)$.

In [ ]:
mask = rater1 != rater2

r1d = rater1[mask]
r2d = rater2[mask]

bins = np.array(list(zip(values - width, values + width))).flatten()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].hist(r1d, bins=bins, color="C0")
axes[0].set_title("Rater 1")
axes[0].set_xticks(values)
axes[0].set_xlabel("Rating")
axes[0].set_ylabel("Frequency")
axes[1].hist(r2d, bins=bins, color="C1")
axes[1].set_title("Rater 2")
axes[1].set_xticks(values)
axes[1].set_xlabel("Rating")
axes[1].set_ylabel("Frequency")
fig.suptitle("Skin Color Ratings on Cases where Raters Disagree")

plt.show()

pd.crosstab(dataframe["rater1"], dataframe["rater2"])


#### Should Referees with very few appearances be excluded from the dataset?

Since we disagreggated the data, an "appearance" is now a single dyad.
Excluding referees with few appearances is generally a good idea,
if we have enough referees with many appearances,
such as to not skew the dataset through outliers. [This notebook](https://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb#We-clean-the-data-by-excluding-interactions-by-refs-who-feature-in-fewer-than-22-dyads)
mentions that referees with fewer than 22 appearances did not play in
one of the major leagues we are interested in, and should thus be removed.

#### What does the distribution of ref occurance look like?

See again [this notebook](https://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb#And-what-does-the-distribution-of-ref-occurance-look-like?).

In [ ]:
all_refs = dataframe.refNum.value_counts()

num_refs = len(all_refs)
print("Total number of referees =", num_refs)
print("Median number of dyads per referee =", np.median(dataframe.refNum.value_counts()))

_, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
axes.hist(dataframe.refNum.value_counts().tolist(), bins=num_refs, edgecolor="black", linewidth=0.3)
axes.set_xscale("symlog") # symetric log scale
axes.set_yscale("symlog")
axes.set_title("Referee occurance, log scaled")
axes.set_xlabel("log (number of occurances)")
axes.set_ylabel("log (frequency)")

plt.show()


#### We clean the data by excluding interactions by refs who feature in fewer than 22 dyads

See again [this notebook](https://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb#We-clean-the-data-by-excluding-interactions-by-refs-who-feature-in-fewer-than-22-dyads).

We keep only about $40\%$ of referees, but more than $97\%$ of dyads.

In [ ]:
good_refs = all_refs[all_refs>21]

print(f"Referees kept: {len(good_refs)} out of {len(all_refs)} ({100 * len(good_refs) / len(all_refs):.2f}%)")
print(f"Dyads kept: {sum(good_refs)} out of {sum(all_refs)} ({100 * sum(good_refs) / sum(all_refs):.2f}%)")

_, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
axes.hist(good_refs.tolist(), num_refs-11, edgecolor="black", linewidth=0.3)
axes.set_xscale("symlog") # symetric log scale
plt.xlim([1,10000])
axes.set_yscale("symlog")
plt.ylim([0,1000])
axes.set_title("Referee occurance following our cull, log scaled")
axes.set_xlabel("log (number of occurances)")
axes.set_ylabel("log (frequency)")

plt.show()

# apply the filter
dataframe = dataframe[dataframe["refNum"].isin(good_refs.index.values)]

#### Should features be normalized and/or centralized?

Centralizing the data is useful for linear regression.


### 2.2 Model Creation

#### Linear Regression

In [ ]:
class LinearRegressor:
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.beta = None
        self.b = None

    def func(self, x):
        if self.beta is None or self.b is None:
            raise RuntimeError("Attempted to use regressor weights without fitting.")
        return self.beta * x + self.b

    def fit(self, assume_centralized=False):
        if not assume_centralized:
            mx, my = self.centralize()
        else:
            mx, my = 0, 0

        if self.x.ndim > 1:
            self.beta = np.linalg.pinv(self.x) @ self.y
            self.b = my - self.beta @ mx
        else:
            self.beta = np.dot(self.x, self.y) / np.sum(np.square(self.x))
            self.b = my - self.beta * mx

        return self.beta, self.b

    def centralize(self):
        mx = np.mean(self.x, axis=0)
        my = np.mean(self.y, axis=0)
        self.x = self.x - mx
        self.y = self.y - my
        return mx, my


def lr_tones(df):
    tones = np.unique(df.skintone.dropna())

    sumgames = []
    sumreds = []

    for tone in tones:
        mask = df.skintone == tone
        games = np.sum(df.games[mask])
        reds = np.sum(df.all_reds[mask])

        sumgames.append(games)
        sumreds.append(reds)

    sumgames = np.array(sumgames)
    sumreds = np.array(sumreds)

    expected = np.sum(sumreds) / np.sum(sumgames)

    lr = LinearRegressor(tones, sumreds / sumgames)
    lr.fit()

    plt.figure(figsize=(14, 8))
    plt.bar(tones, sumreds / sumgames, label="actual", width=0.1, edgecolor="black", lw=0.2)
    plt.axhline(expected, label="expected", linestyle="--", color="black", alpha=0.8)

    left, right = plt.xlim()
    trend_x = np.linspace(left, right, 10)
    plt.plot(trend_x, lr.func(trend_x), label="trend", color="C1")

    plt.xlim(left, right)
    plt.xlabel("Skin Tone Rating (Mean)")
    plt.ylabel("Red Cards per Game")
    plt.legend()
    plt.show()


lr_tones(dataframe)

#### Regression Forest

In [ ]:
class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

In [ ]:
class RegressionTree(Tree):
    def __init__(self):
        super(RegressionTree, self).__init__()
        
    def train(self, data, labels, n_min=100):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            #print(n)
            if n >= n_min:
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                selected_indices = np.random.choice(D, size=D_try, replace=False)
                children = make_regression_split_node(node, selected_indices)
                if children is None:
                    make_regression_leaf_node(node)
                else:
                    stack.extend(children)
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_regression_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response

In [ ]:
def rolling_mean(x, n):
    """ see https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-numpy-scipy """
    return np.convolve(x, np.ones(n), mode="valid") / n

def make_regression_split_node(node, feature_indices):
    """
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature
                     indices to be considered in the present split
    """
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        data_unique = np.sort(np.unique(node.data[:, j]))
        
        # all points have the same j-th value => can't split
        if len(data_unique) == 1:
            continue
        
        # Compute candidate thresholds
        tj = rolling_mean(data_unique, 2)
        
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            mask = node.data[:, j] < t
            # left:
            features_left = node.data[mask]
            labels_left = node.labels[mask]
            lr_left = LinearRegressor(features_left, labels_left)
            popt_left = lr_left.fit()
            response_left = np.dot(features_left, popt_left[0]) + popt_left[1]
            average_response_left = response_left.sum() / features_left.shape[0]
            error_left = ((labels_left - average_response_left)**2).sum()
            
            
            # Right:
            features_right = node.data[~mask]
            labels_right = node.labels[~mask]
            lr_right = LinearRegressor(features_right, labels_right)
            popt_right = lr_right.fit()
            response_right = np.dot(features_right, popt_right[0]) + popt_right[1]
            average_response_right = response_right.sum() / features_right.shape[0]
            error_right = ((labels_right - average_response_right)**2).sum()
            
            
            
            error = error_left + error_right
            
            # choose the best threshold that
            if error < e_min:
                e_min = error
                j_min = j
                t_min = t

                
    if t_min is None:
        return None
        
        
    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    mask = node.data[:, j_min] < t_min
    
    left.data = node.data[mask] # data in left node
    left.labels = node.labels[mask] # corresponding labels
    right.data = node.data[~mask]
    right.labels = node.labels[~mask]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [ ]:
def make_regression_leaf_node(node):
    """
    node: the node to become a leaf
    """
    # compute and store leaf response
    node.N = node.data.shape[0]
    
    lr = LinearRegressor(node.data, node.labels)
    popt = lr.fit()
    response = np.dot(node.data, popt[0]) + popt[1]
    node.response = response.sum() / node.data.shape[0]


In [ ]:
class RegressionForest:
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [RegressionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=100):
        N = data.shape[0]
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            training_set_indices = np.random.choice(N, size=N, replace=True)
            tree.train(data[training_set_indices], labels[training_set_indices])

    def predict(self, x):
        # compute the ensemble prediction
        return np.mean([tree.predict(x) for tree in self.trees])

#### Cross-Validation Test

In [ ]:
x = np.stack([dataframe.skintone.values], axis=1)
y = dataframe.all_reds.values


mask = (np.all(~np.isnan(x), axis=1)) & (~np.isnan(y))
#mask = (~np.isnan(x)) & (~np.isnan(y))
x = x[mask]
y = y[mask]

In [ ]:
from sklearn.model_selection import cross_val_score

class LinearRegressor_Test():
    def __init__(self, popt = None):
        self.popt = popt
    
    def fit(self, X_train, Y_train):
        lr = LinearRegressor(X_train, Y_train)
        self.popt = lr.fit()
        
    def score(self, X_test, Y_test):
        response = np.dot(X_test, self.popt[0]) + self.popt[1]
        error = ((Y_test - response)**2).sum() / Y_test.shape[0]
        return 1 - error
    
    def get_params(self, deep=False):
        return {"popt": self.popt}
    
lr_test = LinearRegressor_Test()
scores = cross_val_score(lr_test, x, y, cv=10)
print(scores)
print(f"score = {np.mean(scores):0.3f} +- {np.std(scores):0.3f}")
print(f"error = {1-np.mean(scores):0.3f} +- {np.std(scores):0.3f}")


In [ ]:
from sklearn.model_selection import cross_val_score

class RegressionForest_Test():
    def __init__(self, forest = None):
        self.forest = forest
    
    def fit(self, X_train, Y_train):
        self.forest = RegressionForest(10)
        self.forest.train(X_train, Y_train)
        
    def score(self, X_test, Y_test):
        response = [self.forest.predict(x) for x in X_test]
        error = ((Y_test - response)**2).sum() / Y_test.shape[0]
        return 1 - error
    
    def get_params(self, deep=False):
        return {"forest": self.forest}
    
rf_test = RegressionForest_Test()
scores = cross_val_score(rf_test, x, y, cv=10)
print(scores)
print(f"score = {np.mean(scores):0.3f} +- {np.std(scores):0.3f}")
print(f"error = {1-np.mean(scores):0.3f} +- {np.std(scores):0.3f}")


### 2.3 Answering the Research Question

From this test, we cannot conclude that
there is a skin color bias in red card
decisions, since these datasets do not
exhibit higher test errors than the original
unscattered dataset.

In [ ]:
skintone_original = np.stack([dataframe.skintone.values], axis=1)
reds_original = dataframe.all_reds.values
mask = (np.all(~np.isnan(skintone_original), axis=1)) & (~np.isnan(reds_original))
skintone_original = skintone_original[mask]
reds_original = reds_original[mask]

for i in range(19):
    skintone = skintone_original[np.random.permutation(skintone_original.shape[0])]

    lr_test = LinearRegressor_Test()
    scores = cross_val_score(lr_test, skintone, reds_original, cv=10)
    print(f"{i+1}\tscore = {np.mean(scores):0.3f} +- {np.std(scores):0.3f}")

    

In [ ]:
skintone_original = np.stack([dataframe.skintone.values], axis=1)
reds_original = dataframe.all_reds.values
mask = (np.all(~np.isnan(skintone_original), axis=1)) & (~np.isnan(reds_original))
skintone_original = skintone_original[mask]
reds_original = reds_original[mask]

for i in range(19):
    skintone = skintone_original[np.random.permutation(skintone_original.shape[0])]

    lr_test = RegressionForest_Test()
    scores = cross_val_score(lr_test, skintone, reds_original, cv=10)
    print(f"{i+1}\tscore = {np.mean(scores):0.3f} +- {np.std(scores):0.3f}")

    

### 2.4 How to Lie with Statistics

As we have already seen in the previous exercise parts, it is possible to
arrive at different conclusions for the dataset depending on which method of
data examination is used. Looking at the total distribution leads to a clear
(if small) correlation between skin color and red card decisions, whereas the
permutation test tells us there is no significant bias.

We have tried several different data cleaning techniques and data set transformations,
but were unable to find one that yields a better distinction between conclusions than
these two tests.

### 2.5 Alternative Hypotheses

#### Reverse Causation: Dark skinned players generate more red card decisions.

Suppose players with dark skin exhibit a more aggressive playstyle overall, which leads
to more fouls made. It would be logical for referees to give more red cards to players
who foul more.

Testing this hypothesis is superfluous as its correlation is equivalent to the original
hypothesis. Since we can only make judgments about correlation, we cannot determine
if the original hypothesis or this one is "more correct".

#### Third Cause Fallacy: Both Dark Skin and Red Card Decisions are caused by Hot Weather

Suppose referees really hate playing in hot weather, since it is more exhausting, thus
giving more red cards because they are annoyed by the weather already.
Since countries with hot weather typically have more people with darker skin, more
red cards are given to darker skinned players as a result.

To test this hypothesis, let's look at the distributions in England (cold weather)
vs. Spain (warm weather).

In [ ]:
mask_england = dataframe.leagueCountry == 0
mask_spain = dataframe.leagueCountry == 3

dataframe_england = dataframe[mask_england]
dataframe_spain = dataframe[mask_spain]

def analysis(df):
    print("Players by skintone:")
    vc = df.skintone.value_counts()
    plt.bar(vc.index, vc.values, width=0.1)
    plt.show()

    print(df.skintone.value_counts())
    print("Total Red Cards received:")
    print(np.sum(df.all_reds))
    print("Total Games played:")
    print(np.sum(df.games))
    print("Red Cards per game:")
    print(np.sum(df.all_reds) / np.sum(df.games))

    print("Red Cards by Skin Tone:")
    print(pd.crosstab(df.skintone, df.all_reds))

analysis(dataframe_england)
analysis(dataframe_spain)


We can see there are more dark skinned players in Spain, and more red cards are given per game,
which supports this hypothesis.

Let's also look at the red card distributions for each country.

In [ ]:
lr_tones(dataframe_england)
lr_tones(dataframe_spain)

There is clear bias shown in England whereas there is little to no bias in Spain. That said,
this analysis is incomplete as a permutation test would be needed here to check for bias.

In [68]:
skintone_original = np.stack([dataframe.skintone.values], axis=1)
reds_original = dataframe.all_reds.values
mask = (np.all(~np.isnan(skintone_original), axis=1)) & (~np.isnan(reds_original))
skintone_original = skintone_original[mask]
reds_original = reds_original[mask]

for i in range(19):
    skintone = skintone_original[np.random.permutation(skintone_original.shape[0])]

    lr_test = RegressionForest_Test()
    scores = cross_val_score(lr_test, skintone, reds_original, cv=10)
    print(f"{i+1}\tscore = {np.mean(scores):0.3f} +- {np.std(scores):0.3f}")

    

1	score = 0.992 +- 0.001
2	score = 0.992 +- 0.001
3	score = 0.992 +- 0.001
4	score = 0.992 +- 0.001
5	score = 0.992 +- 0.001
6	score = 0.992 +- 0.001
7	score = 0.992 +- 0.001
8	score = 0.992 +- 0.001
9	score = 0.992 +- 0.001
10	score = 0.992 +- 0.001
11	score = 0.992 +- 0.001
12	score = 0.992 +- 0.001
13	score = 0.992 +- 0.001
14	score = 0.992 +- 0.001
15	score = 0.992 +- 0.001
16	score = 0.992 +- 0.001
17	score = 0.992 +- 0.001
18	score = 0.992 +- 0.001
19	score = 0.992 +- 0.001
